In [2]:
import numpy as np
from mha021 import *

In [ ]:
# Problem 1

# Define inputs
L = 2 # [m]
h = 2e-3 # [m]
r = 30e-3 # [m]
G = 70e9 # [Pa]
T0 = 300 # [Nm]
Ip = 2 * np.pi * r**3 * h # [m^4]

print('c)')
Le = L / 2

K = G * Ip / Le * np.array([
    [1, -1, 0],
    [-1, 2, -1],
    [0, -1, 1]
])\

f = np.zeros((3, 1))

f[1-1] = T0

bc_dofs = [3]
bc_vals = [0]

a, r = solve_eq(K, f, bc_dofs, bc_vals)
print(a)


print('d)')

s = 3e4 # [Nm]

K = G * Ip / Le * np.array([
    [1, -1, 0],
    [-1, 2, -1],
    [0, -1, 1]
])\
    +\
        np.array([
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, s]
])

f = np.zeros((3, 1))

f[1-1] = T0

a = np.linalg.solve(K, f)
print(a)

a)
[0.02526269 0.01263134 0.        ]
b)
[[0.03526269]
 [0.02263134]
 [0.01      ]]


In [ ]:
# Problem 2

# Define inputs
T0 = 0 # [C]
k = 1 # [W / (m C)]
h = 1 # [W / m^3]
t = 1 # [m]
Lx = 1 # [m]
Ly = 1.25 # [m]
qn = 0 # on symmetry boundaries

import scipy.io
mesh = scipy.io.loadmat('mesh_data.mat') 

Coord = mesh['Coord']                         # [x, y] coords for each node
Dofs = mesh['Dofs']                           
Edof = mesh['Edof']                           # [element number, dof1, dof2, dof3]
Ex = mesh['Ex']                                
Ey = mesh['Ey']                               
right_dofs = mesh['right_dofs']                                                   
top_dofs = mesh['top_dofs']  
left_dofs = mesh['left_dofs']                                                   
bottom_dofs = mesh['bottom_dofs']  

print('d)')

D = k * np.eye(3)

num_nodes = mesh.nodes.shape[0]
num_el = mesh.elements.shape[0]
num_dofs = num_nodes

K = np.zeros((num_dofs, num_dofs))
f = np.zeros((num_dofs))

for el in range(num_el):
    nodes = mesh.nodes[mesh.elements[el] - 1]
    dofs = mesh.edofs[el, :]
    
    Ke, fe = flow2t_Ke_fe(nodes, t, D, Q=h)
    assem(K, Ke, dofs)
    assem(f, fe, dofs)

"""
def fe_edge(nodes, qn, t):
    x1, y1 = nodes[0]
    x2, y2 = nodes[1]
    L = np.sqrt((x2 - x1)**2 + (y2 - y1)**2) 
    fe_ed = t * qn * L / 2 * np.array([[1], [1]])
    return fe_ed
    
left_edges = mesh.edges['inner']
num_edges = len(left_edges)

for egde in range(len_edges):

    edge_nodes = left_edges[edge : edge + 2] - 1
    nodes = mesh.nodes[edge_nodes, :]
    dofs = mesh.edof[nodes + 1]
    
    fe = fe_edges(nodes, qn, t)
    assem(f, fe, dofs)
"""

bc_vals_right = np.zeros_like(right_dofs)
bc_vals_top = np.zeros_like(top_dofs)

bc_dofs = np.hstack([right_dofs, top_dofs])
bc_vals = np.hstack([bc_vals_right, bc_vals_top])

a, r = solve_eq(K, f, bc_dofs, bc_vals)

# Element 18
xe = Ex[el - 1, :]
ye = Ey[el - 1, :]

x = 0.650
y = 0.375

dofs_e = Edof[el - 1, :]
a_e = a[dofs_e - 1]

Ae = ((xe[1] * ye[2] - xe[2] * ye[1]) - 
      (xe[0] * ye[2] - xe[2] * ye[0]) + 
      (xe[0] * ye[1] - xe[1] * ye[0])) / 2

Ne = np.array([
    xe[1] * ye[2] - xe[2] * ye[1] + (ye[1] - ye[2]) * x + (xe[2] - xe[1]) * y,
    xe[2] * ye[0] - xe[0] * ye[2] + (ye[2] - ye[0]) * x + (xe[2] - xe[0]) * y,
    xe[0] * ye[1] - xe[1] * ye[0] + (ye[0] - ye[1]) * x + (xe[1] - xe[0]) * y
])/(2 * Ae)

T_el18 = Ne @ a_e

print(a_e)


d)


AttributeError: 'dict' object has no attribute 'elements'

In [ ]:
# Problem 3

print('a)')